In [ ]:
ACCESS_TOKEN = "MLY|25902363069364084|16956959e4a586fcff63f85b954b528e"
SEQUENCE_FILE = "sequence_id_list.txt"

import os
import json
import requests
import time

Function: For ONE sequence id, collecting all image ids

In [ ]:
def get_image_ids_for_seq(seq_id):
    url = "https://graph.mapillary.com/image_ids"
    params = {
        "access_token": ACCESS_TOKEN,
        "sequence_id": seq_id,
        "limit": 1000
    }

    all_ids = []
    while True:
        r = requests.get(url, params=params)
        if r.status_code != 200:
            print("❌ error", r.status_code, r.text)
            return all_ids
        data = r.json()
        for obj in data.get("data", []):
            all_ids.append(obj["id"])
        # pagination
        paging = data.get("paging", {})
        after = paging.get("cursors", {}).get("after")
        if not after:
            break
        params["after"] = after
        time.sleep(0.2)
    return all_ids

# Test on one sequence
seq = "fP0gGnszXuVolBTFYy6RZW"
ids = get_image_ids_for_seq(seq)
print("Found image ids:", ids)
print("Count:", len(ids))

Found image ids: ['1363848098681972', '756622827424514', '849088327792685', '1482828449500442', '1996635711124172', '838340545554143', '1248663033956135', '2027656178005459', '840447441875934', '826365603117568', '821976017235104', '1176439241050755', '1563131811534490', '1235370605302730', '1106389918231774', '2043221073171608', '1132095785706702', '1129569055992355', '806888532064244', '696993210133079', '1353910949778960', '797872749885008', '1507114400530362', '822885333679812', '1153382622979038', '1565306274891022', '1788616195117008', '793858723463806', '24766531316363575', '1190683482930970', '679427508182758', '797582259919957', '671032792732069', '794682406889241', '1347438763636987', '844009314641556', '1139234435085886', '684518348061226', '1718073685545325', '1127132729619555', '4404794163129990', '2315453748888358', '1939254513682217', '1515449562852630', '2568057170241768', '1593041905399588', '711063514719622', '838756341883577', '791627817077764', '775314365503607', '1

Function: For one sequence ID, using image id list, getting metadata of computed compass angle, compass angle, and computed geometry for each image id in sequence.

In [3]:
def get_image_metadata(image_id):
  url = f"https://graph.mapillary.com/{image_id}"
  params = {
      "access_token": "MLY|25902363069364084|16956959e4a586fcff63f85b954b528e",
      "fields": "compass_angle, computed_compass_angle, computed_geometry, sequence"
  }

  r = requests.get(url, params=params)
  if r.status_code != 200:
    print("Error X")
    return None
  return r.json()


Function: Downloading images from a list of image ids.

In [ ]:
def download_images(image_ids, sequence):
  output_folder = f"./dataset/{sequence}"
  os.makedirs(output_folder, exist_ok=True)
  metadata_list = []

  for img_id in image_ids:
      # get image metadata (to find download URL)
      url = f"https://graph.mapillary.com/{img_id}"
      params = {
          "access_token": ACCESS_TOKEN,
          "fields": "thumb_original_url"  # or 'thumb_1024_url', or 'computed_geometry'
      }
      res = requests.get(url, params=params)
      data = res.json()

      img_url = data.get("thumb_original_url")
      if img_url:
          img_data = requests.get(img_url, stream=True)
          path = os.path.join(output_folder, f"{img_id}.jpg")
          with open(path, "wb") as f:
              for chunk in img_data.iter_content(8192):
                  f.write(chunk)
          print(f"✅ saved {img_id}.jpg")
          metadata_list.append(get_image_metadata(img_id))
      else:
          print(f"⚠️ no image URL found for {img_id}")

  print(f"\n✨ Done! All images saved to: {output_folder}")
  return metadata_list

Hence, we can write code where we go through each sequence id from our sequence_id_list.txt file and collect the image ids, metadata for each image id, and downloading the images too.

In [ ]:
# making list of all sequences from file
with open('sequence_id_list.txt', 'r') as f:
  sequence_ids = f.read()

sequence_ids = json.loads(sequence_ids)
# print(sequence_ids[0])
seq_x_img_id_dict = {}

for i in sequence_ids:
  ids = get_image_ids_for_seq(i)
  seq_x_img_id_dict[i] = ids
  # print("done")
# dictionary containing all sequences corresponding to all image ids in that sequence
print(seq_x_img_id_dict)



Main: Downloading all images for each sequence

In [ ]:
#count = 0     # [ for limiting number of sequences to be downloaded ]
for i in seq_x_img_id_dict:
  metadata = download_images(seq_x_img_id_dict[i], i)
  print(metadata)
  filename = f"./dataset/{i}/metadata.json"
  with open(filename, 'w') as f:
    json.dump(metadata, f, indent = 4)
  #count += 1
  #if count == 1:
    #break